In [65]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from rdkit import Chem
from rdkit.Chem import Descriptors, Draw
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, f1_score, recall_score, precision_score
from imblearn.over_sampling import SMOTE, RandomOverSampler
from xgboost import XGBClassifier
import warnings
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from itertools import product
from sklearn.metrics import classification_report
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from mordred import Calculator, descriptors

#For reproducibility
np.random.seed(42)

#Load the data
file_path = r"C:\Users\20201527\OneDrive - TU Eindhoven\Desktop\8CC00\Group assignement\tested_molecules.csv"
data = pd.read_csv(file_path)
data_all = data
data = data.drop(columns=['PKM2_inhibition', 'ERK2_inhibition'])

In [66]:
# Function to calculate descriptors
def calculate_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [None] * len(Descriptors.descList)
    return [func(mol) for name, func in Descriptors.descList]

# Calculate descriptors for all SMILES
descriptor_names = [name for name, func in Descriptors.descList]
descriptor_values = data['SMILES'].apply(calculate_descriptors)

# Create a DataFrame with descriptor values
descriptor_df = pd.DataFrame(descriptor_values.tolist(), columns=descriptor_names)

descriptor_df

,MaxEStateIndex,MinEStateIndex,MaxAbsEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,NumRadicalElectrons,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,5.313889,0.120833,5.313889,0.120833,0.592228,151.173,142.101,151.085795,58,0,...,0,0,0,0,0,0,0,0,0,0
1,11.238954,-0.366756,11.238954,0.225308,0.785414,250.681,239.593,250.039672,88,0,...,0,0,0,0,0,0,0,0,0,0
2,11.090706,-0.049610,11.090706,0.049610,0.581062,210.277,192.133,210.136828,84,0,...,0,0,0,0,0,0,0,0,0,0
3,11.892238,-0.457824,11.892238,0.076632,0.441090,232.239,220.143,232.084792,88,0,...,0,0,0,0,0,0,0,0,0,0
4,11.693580,-0.498260,11.693580,0.012315,0.720343,226.663,215.575,226.050905,80,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,12.955843,-0.290408,12.955843,0.016627,0.764433,362.385,344.241,362.126657,136,0,...,0,0,0,0,0,0,0,0,0,0
1112,12.983770,-3.772852,12.983770,0.009487,0.594812,374.442,361.338,374.042773,126,0,...,0,1,0,0,0,1,0,0,0,0
1113,10.579691,0.249681,10.579691,0.249681,0.753119,355.849,335.689,355.132015,128,0,...,0,0,0,0,0,0,0,0,0,0
1114,4.812249,0.983396,4.812249,0.983396,0.738254,322.437,304.293,322.125218,116,0,...,0,0,0,0,0,1,0,0,0,0


In [67]:
def check_all_thresholds(constant_bits, threshold_finger_prints, threshold_descriptors):
    np.random.seed(42)
    def smiles_to_fingerprint(smiles, nBits=constant_bits):
        mol = Chem.MolFromSmiles(smiles)
        if mol:
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=nBits)
            return np.array(fp)
        else:
            return np.zeros((nBits,))

    # Apply the function to your dataset with nBits=1024
    fingerprints = data['SMILES'].apply(lambda x: smiles_to_fingerprint(x, nBits=constant_bits))
    fingerprints_matrix = np.stack(fingerprints.values)
    
    # Standardize the fingerprints
    scaler = StandardScaler()
    scaled_fingerprints = scaler.fit_transform(fingerprints_matrix)
    
    # Perform PCA
    pca = PCA()
    pca.fit(scaled_fingerprints)
    
    # Determine the number of components to retain 90% variance
    cumulative_variance = pca.explained_variance_ratio_.cumsum()
    num_components = (cumulative_variance < threshold_finger_prints).sum() + 1
    
    # Transform the dataset using these components
    pca = PCA(n_components=num_components)
    principal_components = pca.fit_transform(scaled_fingerprints)
    
    # Create a DataFrame with the principal components
    pca_df = pd.DataFrame(data=principal_components)

    # Combine PCA and descriptors DataFrames
    combined_df = pd.concat([descriptor_df, pca_df], axis=1)
    combined_df.columns = combined_df.columns.astype(str)

        # Standardize the combined data
    scaler = StandardScaler()
    scaled_combined_data = scaler.fit_transform(combined_df)
    
    # Perform PCA
    pca = PCA()
    pca.fit(scaled_combined_data)
    
    # Determine the number of components to retain 90% variance
    cumulative_variance = pca.explained_variance_ratio_.cumsum()
    num_components = (cumulative_variance < threshold_descriptors).sum() + 1
    
    # Transform the dataset using these components
    pca = PCA(n_components=num_components)
    principal_components = pca.fit_transform(scaled_combined_data)

    final_pca_df = pd.DataFrame(data=principal_components)
    final_pca_df.columns = final_pca_df.columns.astype(str)
    
    return final_pca_df


In [68]:
def check_with_model(descriptor_df):
    np.random.seed(42)
    # Assuming your descriptor_df is already prepared
    X = descriptor_df.drop(columns=['PKM2_inhibition', 'ERK2_inhibition'])
    y1 = descriptor_df['PKM2_inhibition']
    y2 = descriptor_df['ERK2_inhibition']
    y = pd.concat([y1, y2], axis=1)
    
    # Split the data into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=42)
    
    # Standardizing the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    # Ignore all warnings
    warnings.filterwarnings("ignore")
    
    # Apply RandomOverSampler to handle imbalance for each target separately
    ros_pkm2 = RandomOverSampler(random_state=42)
    X_train_ros_pkm2, y_train_ros_pkm2 = ros_pkm2.fit_resample(X_train, y_train['PKM2_inhibition'])
    
    ros_erk2 = RandomOverSampler(random_state=42)
    X_train_ros_erk2, y_train_ros_erk2 = ros_erk2.fit_resample(X_train, y_train['ERK2_inhibition'])
    
    # Calculate scale_pos_weight for each target
    scale_pos_weight_pkm2 = y_train_ros_pkm2.value_counts()[0] / y_train_ros_pkm2.value_counts()[1]
    scale_pos_weight_erk2 = y_train_ros_erk2.value_counts()[0] / y_train_ros_erk2.value_counts()[1]
    
    # Create and train the XGBClassifier for PKM2_inhibition
    model_pkm2 = XGBClassifier(scale_pos_weight=scale_pos_weight_pkm2, random_state=42)
    model_pkm2.fit(X_train_ros_pkm2, y_train_ros_pkm2)
    
    # Create and train the XGBClassifier for ERK2_inhibition
    model_erk2 = XGBClassifier(scale_pos_weight=scale_pos_weight_erk2, random_state=42)
    model_erk2.fit(X_train_ros_erk2, y_train_ros_erk2)
    
    # Predict on the validation set
    y_pred_pkm2 = model_pkm2.predict(X_val)
    y_pred_erk2 = model_erk2.predict(X_val)
    
    # Convert y_val to numpy array for comparison
    y_val_binary = y_val.to_numpy()
    
    # Calculate the accuracy, precision, recall, F1 score, and balanced accuracy for each target column
    accuracy_pkm2 = accuracy_score(y_val_binary[:, 0], y_pred_pkm2)
    accuracy_erk2 = accuracy_score(y_val_binary[:, 1], y_pred_erk2)
    
    precision_pkm2 = precision_score(y_val_binary[:, 0], y_pred_pkm2)
    precision_erk2 = precision_score(y_val_binary[:, 1], y_pred_erk2)
    
    recall_pkm2 = recall_score(y_val_binary[:, 0], y_pred_pkm2)
    recall_erk2 = recall_score(y_val_binary[:, 1], y_pred_erk2)
    
    f1_pkm2 = f1_score(y_val_binary[:, 0], y_pred_pkm2)
    f1_erk2 = f1_score(y_val_binary[:, 1], y_pred_erk2)
    
    # Create a DataFrame to store the results
    results = pd.DataFrame({
        'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
        'PKM2_inhibition': [accuracy_pkm2, precision_pkm2, recall_pkm2, f1_pkm2],
        'ERK2_inhibition': [accuracy_erk2, precision_erk2, recall_erk2, f1_erk2]
    })
    
    # Transpose the DataFrame for the desired format
    results = results.set_index('Metric').T
    
    return results

In [94]:
# Create a DataFrame with the principal components
final_pca_df = check_all_thresholds(512, 0.20, 0.62)

# Save the final PCA dataset
final_pca_df.to_csv('final_pca_dataset.csv', index=False)
final_pca_df['PKM2_inhibition'] = data_all['PKM2_inhibition']
final_pca_df['ERK2_inhibition'] = data_all['ERK2_inhibition']

print("Final PCA dataset with 90% variance saved as 'final_pca_dataset.csv'")

results = check_with_model(final_pca_df)
results

Final PCA dataset with 90% variance saved as 'final_pca_dataset.csv'


Metric,Accuracy,Precision,Recall,F1 Score
PKM2_inhibition,0.980287,0.625000,0.384615,0.476190
ERK2_inhibition,0.940860,0.111111,0.038462,0.057143


In [64]:
# Initialize the search space
values = [512]
thresholds = np.arange(0.05, 1.00, 0.05)

# Placeholder to store the results
best_result = {'PKM2_inhibition': {'f1_score': 0, 'params': ()},
               'ERK2_inhibition': {'f1_score': 0, 'params': ()}}

# Iterate through all combinations of the search space
for val in values:
    for th1 in thresholds:
        for th2 in thresholds:
            try:
                # Create the DataFrame with the principal components
                final_pca_df = check_all_thresholds(val, th1, th2)
                
                # Save the final PCA dataset
                final_pca_df.to_csv('final_pca_dataset.csv', index=False)
                final_pca_df['PKM2_inhibition'] = data_all['PKM2_inhibition']
                final_pca_df['ERK2_inhibition'] = data_all['ERK2_inhibition']
                
                # Evaluate the model
                results = check_with_model(final_pca_df)
                
                # Update the best results for PKM2_inhibition
                if results.loc['PKM2_inhibition', 'F1 Score'] > best_result['PKM2_inhibition']['f1_score']:
                    best_result['PKM2_inhibition']['f1_score'] = results.loc['PKM2_inhibition', 'F1 Score']
                    best_result['PKM2_inhibition']['params'] = (val, th1, th2)
                
                # Update the best results for ERK2_inhibition
                if results.loc['ERK2_inhibition', 'F1 Score'] > best_result['ERK2_inhibition']['f1_score']:
                    best_result['ERK2_inhibition']['f1_score'] = results.loc['ERK2_inhibition', 'F1 Score']
                    best_result['ERK2_inhibition']['params'] = (val, th1, th2)
            except Exception as e:
                print(f"An error occurred with parameters ({val}, {th1}, {th2}): {e}")
                continue

# Output the best results
print(f"Best result for PKM2_inhibition: F1 Score = {best_result['PKM2_inhibition']['f1_score']}, Params = {best_result['PKM2_inhibition']['params']}")
print(f"Best result for ERK2_inhibition: F1 Score = {best_result['ERK2_inhibition']['f1_score']}, Params = {best_result['ERK2_inhibition']['params']}")


Best result for PKM2_inhibition: F1 Score = 0.47058823529411764, Params = (512, 0.2, 0.6000000000000001)
Best result for ERK2_inhibition: F1 Score = 0.1764705882352941, Params = (512, 0.5, 0.55)


In [63]:
thresholds = np.arange(0.05, 1.00, 0.05)
thresholds

array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 , 0.55,
       0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95])